# Tratamento e Limpeza de Dados Meteorológicos

Este notebook realiza o tratamento e limpeza dos dados meteorológicos do INMET.


In [10]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

# Configurações de conexão
POSTGRES_HOST = 'postgres'
POSTGRES_USER = 'postgres'
POSTGRES_PASSWORD = 'postgres'
POSTGRES_DB = 'inmet_db'

engine = create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:5432/{POSTGRES_DB}')


In [11]:
# Carrega dados do PostgreSQL
query = """
SELECT * FROM dados_meteorologicos
ORDER BY datetime
"""

df = pd.read_sql(query, engine)
print(f"Total de registros: {len(df)}")
print(f"Período: {df['datetime'].min()} a {df['datetime'].max()}")
df.head()


Total de registros: 8760
Período: 2021-01-01 00:00:00 a 2021-12-31 23:00:00


,id,datetime,data,hora_utc,precipitacao,pressao,radiacao,temperatura,umidade,vento_direcao,vento_velocidade,created_at
0,1,2021-01-01 00:00:00,2021-01-01,0000,0.0,1013.2,NaN,26.1,77.0,115.0,1.4,2025-11-18 18:50:03.144642
1,2,2021-01-01 01:00:00,2021-01-01,0100,0.0,1013.3,NaN,25.9,79.0,106.0,1.3,2025-11-18 18:50:03.144642
2,3,2021-01-01 02:00:00,2021-01-01,0200,0.0,1012.8,NaN,25.6,81.0,111.0,0.9,2025-11-18 18:50:03.144642
3,4,2021-01-01 03:00:00,2021-01-01,0300,1.2,1012.5,NaN,24.6,88.0,110.0,1.2,2025-11-18 18:50:03.144642
4,5,2021-01-01 04:00:00,2021-01-01,0400,0.0,1011.8,NaN,24.5,90.0,119.0,0.8,2025-11-18 18:50:03.144642


In [12]:
# Verifica valores nulos
print("Valores nulos por coluna:")
print(df.isnull().sum())
print(f"\nPercentual de nulos:")
print((df.isnull().sum() / len(df) * 100).round(2))


Valores nulos por coluna:
id                     0
datetime               0
data                   0
hora_utc               0
precipitacao        1268
pressao             1209
radiacao            4644
temperatura         1209
umidade             1318
vento_direcao       1210
vento_velocidade    1210
created_at             0
dtype: int64

Percentual de nulos:
id                   0.00
datetime             0.00
data                 0.00
hora_utc             0.00
precipitacao        14.47
pressao             13.80
radiacao            53.01
temperatura         13.80
umidade             15.05
vento_direcao       13.81
vento_velocidade    13.81
created_at           0.00
dtype: float64


In [13]:
# Remove registros completamente vazios
df_clean = df.dropna(subset=['temperatura', 'umidade', 'pressao'], how='all')
print(f"Registros após remoção de vazios: {len(df_clean)}")


Registros após remoção de vazios: 7551


In [14]:
# Interpolação de valores faltantes usando método temporal
df_clean = df_clean.sort_values('datetime')

# Interpolação linear para variáveis contínuas
numeric_cols = ['temperatura', 'umidade', 'pressao', 'radiacao', 
                'vento_velocidade', 'vento_direcao', 'precipitacao']

for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].interpolate(method='linear', limit_direction='both')

print("Valores nulos após interpolação:")
print(df_clean.isnull().sum())


Valores nulos após interpolação:
id                  0
datetime            0
data                0
hora_utc            0
precipitacao        0
pressao             0
radiacao            0
temperatura         0
umidade             0
vento_direcao       0
vento_velocidade    0
created_at          0
dtype: int64


In [15]:
# Remove outliers usando IQR para temperatura (variável alvo)
Q1 = df_clean['temperatura'].quantile(0.25)
Q3 = df_clean['temperatura'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Limites de temperatura: {lower_bound:.2f} a {upper_bound:.2f}°C")

outliers = df_clean[(df_clean['temperatura'] < lower_bound) | (df_clean['temperatura'] > upper_bound)]
print(f"Outliers detectados: {len(outliers)}")

# Remove outliers
df_clean = df_clean[(df_clean['temperatura'] >= lower_bound) & (df_clean['temperatura'] <= upper_bound)]
print(f"Registros após remoção de outliers: {len(df_clean)}")


Limites de temperatura: 17.20 a 34.00°C
Outliers detectados: 0
Registros após remoção de outliers: 7551


In [16]:
# Cria features temporais
df_clean['ano'] = df_clean['datetime'].dt.year
df_clean['mes'] = df_clean['datetime'].dt.month
df_clean['dia'] = df_clean['datetime'].dt.day
df_clean['hora'] = df_clean['datetime'].dt.hour
df_clean['dia_semana'] = df_clean['datetime'].dt.dayofweek
df_clean['dia_ano'] = df_clean['datetime'].dt.dayofyear

# Features cíclicas para capturar padrões sazonais
df_clean['hora_sin'] = np.sin(2 * np.pi * df_clean['hora'] / 24)
df_clean['hora_cos'] = np.cos(2 * np.pi * df_clean['hora'] / 24)
df_clean['mes_sin'] = np.sin(2 * np.pi * df_clean['mes'] / 12)
df_clean['mes_cos'] = np.cos(2 * np.pi * df_clean['mes'] / 12)

df_clean.head()


,id,datetime,data,hora_utc,precipitacao,pressao,radiacao,temperatura,umidade,vento_direcao,...,ano,mes,dia,hora,dia_semana,dia_ano,hora_sin,hora_cos,mes_sin,mes_cos
0,1,2021-01-01 00:00:00,2021-01-01,0000,0.0,1013.2,114.6,26.1,77.0,115.0,...,2021,1,1,0,4,1,0.000000,1.000000,0.5,0.866025
1,2,2021-01-01 01:00:00,2021-01-01,0100,0.0,1013.3,114.6,25.9,79.0,106.0,...,2021,1,1,1,4,1,0.258819,0.965926,0.5,0.866025
2,3,2021-01-01 02:00:00,2021-01-01,0200,0.0,1012.8,114.6,25.6,81.0,111.0,...,2021,1,1,2,4,1,0.500000,0.866025,0.5,0.866025
3,4,2021-01-01 03:00:00,2021-01-01,0300,1.2,1012.5,114.6,24.6,88.0,110.0,...,2021,1,1,3,4,1,0.707107,0.707107,0.5,0.866025
4,5,2021-01-01 04:00:00,2021-01-01,0400,0.0,1011.8,114.6,24.5,90.0,119.0,...,2021,1,1,4,4,1,0.866025,0.500000,0.5,0.866025


In [17]:
# Salva dados tratados em nova tabela
df_clean.to_sql('dados_tratados', engine, if_exists='replace', index=False)
print("Dados tratados salvos na tabela 'dados_tratados'")


Dados tratados salvos na tabela 'dados_tratados'


In [18]:
# Estatísticas descritivas
print("Estatísticas descritivas dos dados tratados:")
print(df_clean[['temperatura', 'umidade', 'pressao', 'radiacao', 'vento_velocidade']].describe())


Estatísticas descritivas dos dados tratados:
       temperatura      umidade      pressao     radiacao  vento_velocidade
count  7551.000000  7551.000000  7551.000000  7551.000000       7551.000000
mean     25.736551    81.053569  1012.728010   829.588962          1.374785
std       2.762130    13.599350     2.224171  1095.859425          0.829320
min      18.800000    40.000000  1006.500000     0.000000          0.100000
25%      23.500000    70.000000  1011.100000    26.206410          0.600000
50%      25.500000    82.000000  1012.500000    98.508333          1.300000
75%      27.700000    94.000000  1014.200000  1608.600000          2.100000
max      33.800000   100.000000  1020.500000  4360.100000          3.800000
